In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.head()

In [ ]:
dataset.columns

In [ ]:
# Create the one-hot encoded columns and add them to the DataFrame
dataset = pd.get_dummies(dataset, columns=['sex'] ,dtype = int)

print("\nDataFrame After One-Hot Encoding:")
print(dataset)

In [ ]:
# Create the one-hot encoded columns and add them to the DataFrame
dataset = pd.get_dummies(dataset, columns=['region'] ,dtype = int)

print("\nDataFrame After One-Hot Encoding:")
print(dataset)

In [ ]:
dataset.columns

In [ ]:
import seaborn as sns


In [ ]:
for columns in dataset.columns:
  print("Graph for {columns}")
  plt.figure(figsize=(8, 6))
  sns.scatterplot(data=dataset , x= dataset['expenses'] , y = dataset[columns] , s = 100)
  plt.show()

In [ ]:
labels = dataset.pop('expenses')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dict(dataset), labels))

In [ ]:
dataset = dataset.shuffle(len(dataset))

In [ ]:
train_split = 0.8
train_size = int(train_split * len(dataset))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [ ]:
train_dataset = train_dataset.batch(32)
test_dataset = test_dataset.batch(32)

In [ ]:
# Set a random seed for reproducibility
tf.random.set_seed(42)

# 1. Create the model using the Sequential API
model = tf.keras.Sequential([
  # A dense layer with 1 neuron and an input shape of 1
  tf.keras.layers.Dense(units=1, input_shape=[1], name="output_layer")
])

# 2. Compile the model
model.compile(loss=tf.keras.losses.mae, # Use Mean Absolute Error for loss
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), # Use the Adam optimizer
              metrics=["mae"])

# 3. Get a summary of the model
model.summary()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
